<a href="https://colab.research.google.com/github/DongGwan0505/MNIST-classification-project/blob/main/MNIST_dataset_classification_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Colab 환경 준비
import sys
import tensorflow as tf
import sklearn

print("Python:", sys.version)
print("TensorFlow:", tf.__version__)
print("Scikit-learn:", sklearn.__version__)

if not tf.config.list_physical_devices('GPU'):
    print("⚠️ GPU가 감지되지 않았습니다. Colab 상단 메뉴에서 `런타임 > 런타임 유형 변경 > GPU` 선택하세요.")

In [ ]:
#라이브러리 불러오기
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from functools import partial

In [ ]:
#데이터셋 전처리

# MNIST 로드
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()

X_train, X_valid = X_train_full[:50000], X_train_full[50000:]
y_train, y_valid = y_train_full[:50000], y_train_full[50000:]

# 정규화
X_train, X_valid, X_test = X_train / 255.0, X_valid / 255.0, X_test / 255.0

# 차원 추가 (채널=1)
X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test  = X_test[..., np.newaxis]

# One-hot 인코딩
y_train = keras.utils.to_categorical(y_train, 10)
y_valid = keras.utils.to_categorical(y_valid, 10)
y_test  = keras.utils.to_categorical(y_test, 10)

In [ ]:
#데이터 증강
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1
)
datagen.fit(X_train)

In [ ]:
#CNN 모델 정의
DefaultConv2D = partial(Conv2D, kernel_size=3, padding="SAME", kernel_initializer="he_normal")

model = keras.models.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[28, 28, 1]),
    BatchNormalization(), keras.layers.Activation("relu"),
    MaxPooling2D(pool_size=2),

    DefaultConv2D(filters=128), BatchNormalization(), keras.layers.Activation("relu"),
    DefaultConv2D(filters=128), BatchNormalization(), keras.layers.Activation("relu"),
    MaxPooling2D(pool_size=2),

    DefaultConv2D(filters=256), BatchNormalization(), keras.layers.Activation("relu"),
    DefaultConv2D(filters=256), BatchNormalization(), keras.layers.Activation("relu"),

    GlobalAveragePooling2D(),

    Dense(128, activation="relu"), Dropout(0.5),
    Dense(64, activation="relu"), Dropout(0.5),
    Dense(10, activation="softmax")
])
model.summary()

In [ ]:
#콜백 정의 & 학습
checkpoint_cb = keras.callbacks.ModelCheckpoint("best_model.keras", save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
reduce_lr_cb = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-5)

model.compile(loss="categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    epochs=100,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb, early_stopping_cb, reduce_lr_cb])

In [ ]:
#성능 평가
test_loss, test_acc = model.evaluate(X_test, y_test)
print("✅ 최종 Test Accuracy:", test_acc)